<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/auduvignac/Deep-learning/blob/main/Projet/importGolois2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

In [1]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2025.zip
!unzip project2025.zip
!ls -l

In [2]:
!pip install tensorrt-bindings==8.6.1
!pip install --extra-index-url https://pypi.nvidia.com tensorrt-libs
!pip install tensorflow[and-cuda]==2.15.0

In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import gc

import golois

planes = 31
moves = 361
N = 10000
epochs = 20
batch = 128
filters = 32

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("Tensorflow version", tf.__version__)
print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)


input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
for _ in range (5):
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)
value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary ()

model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

for i in range (1, epochs + 1):
    print(f'epoch {str(i)}')
    golois.getBatch (input_data, policy, value, end, groups, i * N)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value},
                        epochs=1, batch_size=batch)
    if (i % 5 == 0):
        gc.collect ()
    if (i % 20 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
        model.save ('test.h5')

In [ ]:
from google.colab import files
files.download('test.h5')

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import gc
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

import golois

class GONet():

  def __init__(self, batch = 128, epochs = 20, filters = 32, moves = 361, N = 10000, planes = 31):
    self.batch = batch
    self.epochs = epochs
    self.filters = filters
    self.moves = moves
    self.N = N
    self.planes = planes
    self.set_input_data(N, planes)
    self.set_policy(N, moves)
    self.set_value(N)
    self.set_end(N)
    self.set_groups(N)
    print ("Tensorflow version", tf.__version__)
    print ("getValidation", flush = True)
    golois.getValidation (input_data, policy, value, end)

  def set_input_data(self, N, planes):
    input_data = np.random.randint(2, size=(N, 19, 19, planes))
    self.input_data = input_data.astype ('float32')

  def get_input_data(self):
    return self.input_data

  def set_policy(self, N, moves):
    policy = np.random.randint(moves, size=(N,))
    self.policy = keras.utils.to_categorical (policy)

  def get_policy(self):
    return self.policy

  def set_value(self, N):
    value = np.random.randint(2, size=(N,))
    self.value = value.astype ('float32')

  def get_value(self):
    return self.value

  def set_end(self, N):
    end = np.random.randint(2, size=(N, 19, 19, 2))
    self.end = end.astype ('float32')

  def get_end(self):
    return self.end

  def set_groups(self, N):
    groups = np.zeros((N, 19, 19, 1))
    self.groups = groups.astype ('float32')

  def get_groups(self):
    return self.groups

  def set_model(self):
    input_ = keras.Input(shape=(19, 19, self.planes), name='board')
    x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
    for _ in range (5):
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
    policy_head = layers.Flatten()(policy_head)
    policy_head = layers.Activation('softmax', name='policy')(policy_head)
    value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
    value_head = layers.Flatten()(value_head)
    value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
    value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

    model = keras.Model(inputs=input_, outputs=[policy_head, value_head])

    model.summary()

    model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9),
                  loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
                  loss_weights={'policy' : 1.0, 'value' : 1.0},
                  metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

    self.model = model

  def train_model(self, epochs):
    for i in range (1, epochs + 1):
        print(f'epoch {str(i)}')
        golois.getBatch(self.input_data, self.policy, self.value, self.end, self.groups, i * self.N)
        history = self.model.fit( self.input_data,
                                  {'policy': self.policy, 'value': self.value},
                                  epochs=1,
                                  batch_size=self.batch
                                )
        if (i % 5 == 0):
            gc.collect ()
        if (i % 20 == 0):
            golois.getValidation (self.input_data, self.policy, self.value, self.end)
            val = model.evaluate (self.input_data,
                                  [self.policy, self.value], verbose = 0, batch_size=self.batch)
            print(f"{val=}")
            model.save ('test.h5')

In [ ]:
GONet = GONet()
GONet.set_model()
GONet.train_model(20)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import gc

import golois

planes = 31
moves = 361
N = 10000
epochs = 20
batch = 128
filters = 32

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("Tensorflow version", tf.__version__)
print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)


import tensorflow as tf
from tensorflow.keras import layers, regularizers, Model

# 🟢 Residual Block Function
def residual_block(x, filters):
    """A ResNet-style residual block with skip connection."""
    shortcut = x  # Save input for shortcut connection

    x = layers.Conv2D(filters, (3, 3), padding='same', use_bias=False, 
                      kernel_regularizer=regularizers.l2(0.0001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, shortcut])  # Add residual connection
    x = layers.ReLU()(x)

    return x

# 🟢 Model Definition
filters = 24  # More filters for better feature extraction
planes = 31   # Feature planes

input_layer = layers.Input(shape=(19, 19, planes), name='board')

# Initial Conv Layer
x = layers.Conv2D(filters, (3, 3), padding='same', activation='relu', 
                  kernel_regularizer=regularizers.l2(0.0001))(input_layer)

# 🔷 Apply 2 Residual Blocks
for _ in range(2):
    x = residual_block(x, filters)

# 🔷 Policy Head (Move Probabilities)
policy_head = layers.Conv2D(2, (1, 1), activation='relu', padding='same', 
                            use_bias=False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Dense(361, activation='softmax', name='policy', 
                           kernel_regularizer=regularizers.l2(0.0001))(policy_head)

# 🔷 Value Head (Win Probability)
value_head = layers.Conv2D(1, (1, 1), activation='relu', padding='same', 
                           use_bias=False, kernel_regularizer=regularizers.l2(0.0001))(x)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', 
                          kernel_regularizer=regularizers.l2(0.0001))(value_head)

# 🟢 Create Model
model = Model(inputs=input_layer, outputs=[policy_head, value_head])
model.summary()


model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

for i in range (1, epochs + 1):
    print(f'epoch {str(i)}')
    golois.getBatch (input_data, policy, value, end, groups, i * N)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value},
                        epochs=1, batch_size=batch)
    if (i % 5 == 0):
        gc.collect ()
    if (i % 20 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
        model.save ('test.h5')